<a href="https://colab.research.google.com/github/Emakporpaul/EPL-2024-2025-Premier-League-Analysis/blob/main/deep_learning_lung_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
batch_size = 32
img_height = 180
img_width = 180
epochs = 2

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data_dir = '/content/drive/MyDrive/new/lung dataset'

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)
# allows importing our data from the directory and creates the training subset

Found 13808 files belonging to 2 classes.
Using 11047 files for training.


In [8]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 13808 files belonging to 2 classes.
Using 2761 files for validation.


In [9]:
train_ds.class_names

['covid', 'normal']

In [10]:
val_ds.class_names

['covid', 'normal']

In [11]:
# build DL model from scratch
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [12]:
# create a variable model
model = Sequential(
    [
        layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(img_height, img_width, 3)),

        layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 180, 180, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 180, 180, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 90, 90, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 90, 90, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 45, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 45, 45, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 61952)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,929,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,060,289 (30.75 MB)

 Trainable params: 8,060,289 (30.75 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# compile the data before training
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
epochs=2

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)
# epochs tell you how many times to train the model

Epoch 1/2


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:789: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


346/346 ━━━━━━━━━━━━━━━━━━━━ 3816s 11s/step - accuracy: 0.7762 - loss: 15.0796 - val_accuracy: 0.9120 - val_loss: 0.2473
Epoch 2/2
346/346 ━━━━━━━━━━━━━━━━━━━━ 2428s 7s/step - accuracy: 0.9089 - loss: 0.2352 - val_accuracy: 0.9189 - val_loss: 0.2424


Pre-Trained Models

1. Inception
2. MobileNet
3. Res-Net
4. VGG-16

https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

In [16]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

In [17]:
# initialize an input layer with a pre-trained layer
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

/tmp/ipython-input-3887569063.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [21]:
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 2

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/2
346/346 ━━━━━━━━━━━━━━━━━━━━ 419s 1s/step - accuracy: 0.7870 - loss: 0.4541 - val_accuracy: 0.8881 - val_loss: 0.2838
Epoch 2/2
346/346 ━━━━━━━━━━━━━━━━━━━━ 410s 1s/step - accuracy: 0.8696 - loss: 0.3072 - val_accuracy: 0.8852 - val_loss: 0.2669


In [25]:
from tensorflow.keras.preprocessing import image

# Load the image and preprocess it
def preprocess_image(img_path, img_height, img_width):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)    # Add batch dimension
    print(img_array.shape)
    return img_array

In [27]:
# Path to your image
img_path = '/content/drive/MyDrive/new/lung dataset/covid/COVID-1018.png'

# Preprocess the image
img_array = preprocess_image(img_path, img_height, img_width)

# Make predictions
predictions = model.predict(img_array)
print(predictions)


(1, 180, 180, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[0.71364117]]


In [28]:
# Convert prediction to class label
if predictions[0][0] > 0.5:
    print("Predicted class: 1 (Positive)")
else:
    print("Predicted class: 0 (Negative)")

Predicted class: 1 (Positive)


In [31]:
# Path to your image
img_path = '/content/drive/MyDrive/new/lung dataset/normal/Normal-10049.png'

# Preprocess the image
img_array = preprocess_image(img_path, img_height, img_width)

# Make predictions
predictions = model.predict(img_array)
print(predictions)


(1, 180, 180, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
[[0.95902383]]


In [32]:
# Convert prediction to class label
if predictions[0][0] > 0.5:
    print("Predicted class: 1 (Positive)")
else:
    print("Predicted class: 0 (Negative)")

Predicted class: 1 (Positive)
